In [2]:
from langchain.document_loaders import TextLoader

loader = TextLoader("./data/data.txt")
d = loader.load()
from langchain_ollama.llms import OllamaLLM

In [3]:
from langchain.document_loaders.csv_loader import CSVLoader
# loader = CSVLoader(file_path="movies.csv")
# data = loader.load()
# data

In [4]:
# pip3 install unstructured libmagic python-magic python-magic-bin
from langchain.document_loaders import UnstructuredURLLoader

In [5]:
from langchain_community.document_loaders import WebBaseLoader

# loader = WebBaseLoader(
#        [
#         "https://sslwireless.com/our-company/",
#         "https://sslwireless.com/our-milestones/"
#     ])
# data = loader.load()


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
loader = UnstructuredURLLoader(
    urls = [
        "https://sslwireless.com/our-company/",
        "https://sslwireless.com/our-milestones/"
    ]
)
data = loader.load()

In [7]:
#data[0].page_content
text = ""
for t in data:
    text = text + t.page_content

In [8]:
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size=200,
    chunk_overlap=0
)
# splitter.split_text(text)

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

r_splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n", "\n", " "],  # List of separators based on requirement (defaults to ["\n\n", "\n", " "])
    chunk_size = 200,  # size of each chunk created
    chunk_overlap  = 0,  # size of  overlap between chunks in order to maintain the context
    length_function = len  # Function to calculate size, currently we are using "len" which denotes length of string however you can pass any token counter)
)

chunks = r_splitter.split_documents(data)

a = chunks[0]
a

Document(metadata={'source': 'https://sslwireless.com/our-company/'}, page_content='About us\n\nSSL Wireless is recognized for its rapid development and efficient project execution by working with industry-standard technologies.')

In [10]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./data/Anisur_Rahman_resume_2024.pdf")

d = loader.load()


In [25]:
links = get_all_links("https://sslwireless.com/")
loader = UnstructuredURLLoader(list(links))
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
        separators=['\n\n', '\n', '.', ','],
        chunk_size=1000
    )
docs = text_splitter.split_documents(data)


Base URL is : https://sslwireless.com/
Total Links get:  24


In [12]:
from langchain_huggingface import HuggingFaceEmbeddings
# models = "hkunlp/instructor-large", sentence-transformers/all-MiniLM-L6-v2
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/opt/anaconda3/envs/2024/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/opt/anaconda3/envs/2024/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
from langchain_community.vectorstores import FAISS,Chroma
#docs = [doc for doc in docs] 
vectorstore = FAISS.from_documents(docs, embeddings)
#similar_docs = vectorstore.as_retriever().similarity_search(query, k=5)  # Adjust k as needed

# query = "E-Commerce Businesses"
# similar_docs = vectorstore.similarity_search(query=query, k=1)
# similar_docs


In [18]:
from langchain_groq import ChatGroq
from langchain.chat_models import ChatOpenAI
llm = ChatGroq(
        temperature=0,
        groq_api_key="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",
        model_name="llama-3.1-70b-versatile"
    )
model = "llama3.1:8b"
open_ai_key = ""
#llm = ChatOpenAI(model_name="gpt-4", temperature=0,openai_api_key=open_ai_key)

#llm = OllamaLLM(model=model)

In [15]:
#llm.invoke("what is ai?")

In [26]:
from langchain.chains import RetrievalQAWithSourcesChain

chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorstore.as_retriever())
try:
    query = "Tell me something about SSL wireless"
    result = chain.invoke(query,return_only_outputs=True)
    print(result)
    # r = chain.invoke("hi")
    # r
except Exception as e:
    print(e)

{'answer': 'SSL Wireless is a software development and services company that has been working in the industry for over a decade. They have experience in providing scalable IT solutions, particularly in the Value-Added Services, Banking, Corporate, and FinTech industries of Bangladesh. They have also worked with various international corporations such as Visa, MasterCard, and AWS, and have developed and deployed a Digital Wallet platform called FastPay. Additionally, they have helped launch Mobile VAS and SMS Banking in Bangladesh, which has transformed into mobile financial services today.\n\n', 'sources': ''}


In [17]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

query = "What is the payment gateway name of the sslwareless?"

retrieved_docs = vectorstore.similarity_search(query, k=5)

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="Using the following information, please answer the question:\n\n{context}\n\nQuestion: {question}\nAnswer: if don't get any answer return 'I don't know'"
)

combined_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

formatted_prompt = prompt_template.format(context=combined_context, question=query)

llm_chain = LLMChain(llm=llm, prompt=prompt_template)

response = llm_chain.run(context=combined_context, question=query)
response


/var/folders/y2/pbbl1rcd3kj9crfctyw4fgy80000gn/T/ipykernel_78226/2567600204.py:17: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt_template)
/var/folders/y2/pbbl1rcd3kj9crfctyw4fgy80000gn/T/ipykernel_78226/2567600204.py:19: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm_chain.run(context=combined_context, question=query)


'The payment gateway name of SSL Wireless is SSLCOMMERZ.'

In [22]:
import requests
from bs4 import BeautifulSoup
def get_all_links(base_url):
    print(f"Base URL is : {base_url}")
    try:
        response = requests.get(base_url)
        response.raise_for_status()  # Check for HTTP errors
        soup = BeautifulSoup(response.content, 'html.parser')
        links = set()

        for a_tag in soup.find_all('a', href=True):
            link = a_tag['href']
            # Ensure the link is a full URL
            if link.startswith('/'):
                link = base_url + link
            elif not link.startswith('http'):
                continue
            links.add(link)
        print(f"Total Links get:  {len(links)}")
        return links
    except Exception as e:
        print(f"Error fetching {base_url}: {e}")
        return set()